In [2]:
!pip install pandas
!pip install sentence-transformers
!pip install fairseq
!pip install langchain
!pip install langchain-community

     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ---------------------------------------- 61.0/61.0 kB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   -- ------------------------------------- 0.7/11.6 MB 21.5 MB/s eta 0:00:01
   ----------- ---------------------------- 3.4/11.6 MB 43.6 MB/s eta 0:00:01
   --------------------------- ------------ 8.0/11.6 MB 63.7 MB/s eta 0:00:01
   -------------------------------------- - 11.3/11.6 MB 81.8 MB/s eta 0:00:01
   ---------------------------------------- 11.6/11.6 MB 65.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ------ --------------------------------- 2.7/15.8 MB 84.2 MB/s eta 0:00:01
   ---------------- ----------------------- 6.5/15.8 MB 83.4 MB/s eta 0:00:01
   --------------------------- ------------ 10.7/15.8 MB 81.8 MB/s eta 0:00:01
   -------------------------------------- - 15.2/15.8 MB 93.9 MB/s eta 0:00:01
  

In [67]:
import pandas as pd

from scipy.spatial.distance import cosine


from langchain_core.documents import Document

from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

from langchain_community.vectorstores import FAISS

from langchain_community.vectorstores.utils import DistanceStrategy

from tqdm import tqdm

import numpy as np
import numpy as np
import pandas as pd
import faiss
from transformers import AutoModel, AutoTokenizer
import torch 


   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
   ---------------------------------------- 0.1/14.5 MB 1.7 MB/s eta 0:00:09
   -------- ------------------------------- 3.1/14.5 MB 40.2 MB/s eta 0:00:01
   -------------------- ------------------- 7.3/14.5 MB 58.2 MB/s eta 0:00:01
   ---------------------------------- ----- 12.4/14.5 MB 93.9 MB/s eta 0:00:01
   ---------------------------------------  14.5/14.5 MB 108.8 MB/s eta 0:00:01
   ---------------------------------------- 14.5/14.5 MB 72.5 MB/s eta 0:00:00


DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


### Creating vector spacem

In [112]:
df_list_of_documents = pd.read_excel(r"C:\Users\hongk\Desktop\embedding_test\vector_database.xlsx")
df_testing = pd.read_excel(r"C:\Users\hongk\Desktop\embedding_test\testing.xlsx")


In [121]:
df_testing['words'].tolist()

['Cardiac Catheterization',
 'Defibrillator Implantation',
 'Pacemaker Installation',
 'Cardioversion',
 'Heart Valve Surgery',
 'Coronary Artery Bypass Grafting (CABG)',
 'Endovenous Laser Therapy (EVLT)',
 'Varicose Vein Treatment',
 'Sclerotherapy',
 'Hemorrhoidectomy',
 'Anal Fistula Repair',
 'Colostomy',
 'Ileostomy',
 'Gastrostomy',
 'Laparoscopic Appendectomy',
 'Cholecystectomy',
 'Hernia Repair (Laparoscopic)',
 'Hernia Repair (Open)',
 'Bariatric Surgery (Gastric Bypass)',
 'Bariatric Surgery (Gastric Sleeve)',
 'Bariatric Surgery (LAP-BAND)',
 'Thyroidectomy',
 'Parathyroidectomy',
 'Adrenalectomy',
 'Pituitary Surgery',
 'Brain Tumor Removal',
 'Craniotomy',
 'Deep Brain Stimulation (DBS)',
 'Spinal Cord Stimulator Implantation',
 'Hydrocephalus Shunt Placement',
 'Epilepsy Surgery',
 'Neuropsychological Testing',
 'Cognitive Behavioral Therapy (CBT)',
 'Dialectical Behavior Therapy (DBT)',
 'Electroconvulsive Therapy (ECT)',
 'Transcranial Magnetic Stimulation (TMS)',
 'P

In [116]:
df_list_of_documents

,words
0,Consultation Fee
1,General Practitioner Fee
2,Specialist Consultation Fee
3,Emergency Room Fee
4,Hospital Admission Fee
...,...
506,Copy of Medical Records
507,Insurance Pre-Authorization
508,Billing Inquiry
509,Patient Education Materials


In [66]:
embeddings = HuggingFaceEmbeddings(model_name="jinaai/jina-embeddings-v2-base-en")

2024-06-13 15:20:44 | INFO | sentence_transformers.SentenceTransformer | Use pytorch device_name: cpu
2024-06-13 15:20:44 | INFO | sentence_transformers.SentenceTransformer | Load pretrained SentenceTransformer: jinaai/jina-embeddings-v2-base-en
Some weights of BertModel were not initialized from the model checkpoint at jinaai/jina-embeddings-v2-base-en and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.1.output.dense.weight', 'encoder.layer.10.intermediate.dense.bias', 'encoder.layer.10.intermediate.dens

In [ ]:
def generate_embeddings(texts, tokenizer, model):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).cpu().numpy()

In [195]:
def remove_words_before_slash(text):
  """Removes all words before the first dot (.) in a string.

  Args:
    text: The input string.

  Returns:
    The string with all words before the first dot removed.
  """
  parts = text.split('/')
  if len(parts) > 1:
    return parts[1].strip()
  else:
    return ''

# Example usage
text = "jinaai/jina-embeddings-v2-base-en"
result = remove_words_before_slash(text)
print(result)  # Output: jina-embeddings-v2-base-en

jina-embeddings-v2-base-en


In [210]:
# Step 2: Read words from the CSV file
csv_file = r"C:\Users\hongk\Desktop\embedding_test\vector_database.xlsx" # update with your CSV file path
df_list_of_documents = pd.read_excel(csv_file)
words = df_list_of_documents['words'].tolist()

In [211]:
def vector_db_creation(model_name,xlsx_file):
    # time how long it takes to create the index
    import time
    start = time.time()
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    df_list_of_documents = pd.read_excel(xlsx_file)
    words = df_list_of_documents['words'].tolist()
    # # Step 3: Generate vectors for the words
    vectors = generate_embeddings(words, tokenizer, model)

    # # Step 4: Create the FAISS index
    dimension = vectors.shape[1]  # dimension of vectors (determined by the model)
    index = faiss.IndexFlatIP(dimension)  
    index.add(vectors)
    end = time.time()
    total_time = end - start
    print(f"Time taken to create the index: {end - start:.2f} seconds")
    df_vector_storage_time_helper = pd.DataFrame(columns = ['model_name','total_time'])
    df_vector_storage_time_helper['model_name'] = [model_name]
    df_vector_storage_time_helper['total_time'] = [total_time]
    return index,tokenizer,model,df_vector_storage_time_helper

In [224]:
vector_storage,tokenizer,model,df_vector_storage_time_helper  = vector_db_creation("sentence-transformers/all-MiniLM-L6-v2",r"C:\Users\hongk\Desktop\embedding_test\vector_database.xlsx")

Time taken to create the index: 1.77 seconds


In [223]:
query_word = 'Cardiac Catheterization'
query_vector = generate_embeddings([query_word], tokenizer, model)
k = 5  # number of nearest neighbors to retrieve
D, I = vector_storage.search(query_vector, k)
# # Print results
print("Query word:", query_word)
print("Distances:", D)
print("Indices:", I)
print("Nearest neighbors:")
for idx in I[0]:
    print(df_list_of_documents['words'].iloc[idx])

Query word: Cardiac Catheterization
Distances: [[55.445026 55.026802 54.568367 54.111465 53.71783 ]]
Indices: [[129 316 449 126 237]]
Nearest neighbors:
Vaccination (Rabies)
Coronary Artery Bypass Grafting (CABG)
Pregnancy Test
Vaccination (Shingles)
MRI Scan


In [225]:
query_word = 'Cardiac Catheterization'
query_vector = generate_embeddings([query_word], tokenizer, model)
k = 5  # number of nearest neighbors to retrieve
D, I = vector_storage.search(query_vector, k)
# # Print results
print("Query word:", query_word)
print("Distances:", D)
print("Indices:", I)
print("Nearest neighbors:")
for idx in I[0]:
    print(df_list_of_documents['words'].iloc[idx])

Query word: Cardiac Catheterization
Distances: [[18.330544 18.217361 18.198412 18.198412 16.038517]]
Indices: [[484 316 289  46 370]]
Nearest neighbors:
Cardiac Rehabilitation
Coronary Artery Bypass Grafting (CABG)
Cardiology Consultation
Cardiology Consultation
Bladder Sling Procedure


In [218]:
for idx in I[0]:
    print(df_list_of_documents['words'].iloc[idx])

Query word: Cardiac Catheterization
Distances: [[55.445026 55.026802 54.568367 54.111465 53.71783 ]]
Indices: [[129 316 449 126 237]]
Nearest neighbors:
Vaccination (Rabies)
Coronary Artery Bypass Grafting (CABG)
Pregnancy Test
Vaccination (Shingles)
MRI Scan


In [207]:
list_of_model = ['intfloat/e5-small-v2',
                 'sentence-transformers/all-MiniLM-L6-v2'
                 ,'intfloat/multilingual-e5-large',
                 'sentence-transformers/all-MiniLM-L12-v2',
                 'google-bert/bert-base-uncased',
                 'google-bert/bert-large-uncased',
                 'allenai/longformer-base-4096',
                 'jinaai/jina-embeddings-v2-base-en',
                 #'google/bigbird-roberta-base',
                 'pritamdeka/S-PubMedBert-MS-MARCO',
                 'Charangan/MedBERT',
                 'medicalai/ClinicalBERT',
                 'dmis-lab/biobert-v1.1',
                 'bvanaken/CORe-clinical-outcome-biobert-v1',
                 'microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext',
                 'yikuan8/Clinical-Longformer',
                 #'yikuan8/Clinical-BigBird',
                 #'Falconsai/medical_summarization',
                 'UFNLP/gatortronS']

In [208]:
df_vector_storage_time = pd.DataFrame(columns = ['model_name','total_time'])

In [233]:
df_results = pd.DataFrame(columns = ['model_name','words','cosine_similarity','k','matched_words','time_taken'])

In [219]:
[df_list_of_documents['words'].tolist()[index]]

['Gout Treatment']

In [248]:
list_of_model

['intfloat/e5-small-v2',
 'sentence-transformers/all-MiniLM-L6-v2',
 'intfloat/multilingual-e5-large',
 'sentence-transformers/all-MiniLM-L12-v2',
 'google-bert/bert-base-uncased',
 'google-bert/bert-large-uncased',
 'allenai/longformer-base-4096',
 'jinaai/jina-embeddings-v2-base-en',
 'google/bigbird-roberta-base',
 'pritamdeka/S-PubMedBert-MS-MARCO',
 'Charangan/MedBERT',
 'medicalai/ClinicalBERT',
 'dmis-lab/biobert-v1.1',
 'bvanaken/CORe-clinical-outcome-biobert-v1',
 'microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext',
 'yikuan8/Clinical-Longformer',
 'yikuan8/Clinical-BigBird',
 'Falconsai/medical_summarization',
 'UFNLP/gatortronS']

In [249]:
model

BigBirdModel(
  (embeddings): BigBirdEmbeddings(
    (word_embeddings): Embedding(50358, 768, padding_idx=0)
    (position_embeddings): Embedding(4096, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BigBirdEncoder(
    (layer): ModuleList(
      (0-11): 12 x BigBirdLayer(
        (attention): BigBirdAttention(
          (self): BigBirdSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BigBirdSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=

In [252]:
df_vector_storage_time

,model_name,total_time
0,intfloat/e5-small-v2,3.035501
1,sentence-transformers/all-MiniLM-L6-v2,1.928166
2,intfloat/e5-small-v2,3.262298
3,intfloat/e5-small-v2,2.997717
4,sentence-transformers/all-MiniLM-L6-v2,1.823421
5,intfloat/e5-small-v2,3.063021
6,intfloat/e5-small-v2,3.062507
7,sentence-transformers/all-MiniLM-L6-v2,1.983221
8,intfloat/e5-small-v2,3.234687
9,sentence-transformers/all-MiniLM-L6-v2,1.737268


In [254]:
list_of_model[17:]

['Falconsai/medical_summarization', 'UFNLP/gatortronS']

In [246]:
df_vector_storage_time.to_csv(r"C:\Users\hongk\Desktop\embedding_test\df_vector_storage_timeV2.csv",index = False)

In [255]:
import time
list_of_words = df_testing['words'].tolist()
list_of_words = [x for x in list_of_words if str(x) != 'nan']
output_location = r"C:\Users\hongk\Desktop\embedding_test"
for model_name in list_of_model[18:]:
    vector_storage,tokenizer,model,df_vector_storage_time_helper  = vector_db_creation(model_name,r"C:\Users\hongk\Desktop\embedding_test\vector_database.xlsx")
    df_vector_storage_time = pd.concat([df_vector_storage_time,df_vector_storage_time_helper], ignore_index=True)
    print('Start' + model_name)
    df_results = pd.DataFrame(columns = ['model_name','words','cosine_similarity','k','matched_words','time_taken'])
    for words in list_of_words:
        start = time.time()
        query = generate_embeddings([words], tokenizer, model)
        D, I = vector_storage.search(query, 3)
        end = time.time()
        total_time = end - start
        for i, (distance, index) in enumerate(zip(D[0], I[0])):
            df_results_helper = pd.DataFrame(columns = ['model_name','words','cosine_similarity','time_taken'])
            df_results_helper['model_name'] = [model_name]
            df_results_helper['words'] = [words]
            df_results_helper['k'] = [i+1]
            df_results_helper['matched_words'] = [df_list_of_documents['words'].tolist()[index]]
            df_results_helper['cosine_similarity'] = [distance]
            df_results_helper['time_taken'] = [total_time]
            df_results = pd.concat([df_results,df_results_helper])
    name = remove_words_before_slash(model_name)
    df_results.to_excel(f"{output_location}\\{name}_results.xlsx",index = False)
    print('End' + model_name)
df_vector_storage_time.to_excel(f"{output_location}\\vector_storage_time.xlsx",index = False)

c:\Users\hongk\Desktop\embedding_test\.conda\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\hongk\Desktop\embedding_test\.conda\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hongk\.cache\huggingface\hub\models--UFNLP--gatortronS. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run P

Time taken to create the index: 56.90 seconds
StartUFNLP/gatortronS


C:\Users\hongk\AppData\Local\Temp\ipykernel_12668\3733059328.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_results = pd.concat([df_results,df_results_helper])


EndUFNLP/gatortronS


,model_name,words,cosine_similarity,k,matched_words,time_taken
0,intfloat/e5-small-v2,A,39.541912,1,Vaccination (Rabies),0.012002
0,intfloat/e5-small-v2,A,39.424778,2,Vaccination (Shingles),0.012002
0,intfloat/e5-small-v2,A,38.481697,3,Copy of Medical Records,0.012002
0,intfloat/e5-small-v2,n,46.761600,1,D&C (Dilation and Curettage),0.012003
0,intfloat/e5-small-v2,n,46.688824,2,Vaccination (Rabies),0.012003
...,...,...,...,...,...,...
0,sentence-transformers/all-MiniLM-L6-v2,Cardiac Catheterization,18.217361,2,Coronary Artery Bypass Grafting (CABG),0.008002
0,sentence-transformers/all-MiniLM-L6-v2,Cardiac Catheterization,18.198412,3,Cardiology Consultation,0.008002
0,sentence-transformers/all-MiniLM-L6-v2,Defibrillator Implantation,11.646187,1,Cochlear Implant Surgery,0.007002
0,sentence-transformers/all-MiniLM-L6-v2,Defibrillator Implantation,9.677955,2,Mohs Surgery,0.007002


In [229]:
df_results_helper

,model_name,words,cosine_similarity,time_taken,k,matched_words
0,sentence-transformers/all-MiniLM-L6-v2,Defibrillator Implantation,9.590363,0.007002,3,Coronary Artery Bypass Grafting (CABG)


jina-embeddings-v2-base-en


In [191]:
result

''

In [178]:
name = model_name+'_results'

In [185]:
import os 
os.system(f'start excel.exe {name}.xlsx')

In [179]:
name

'jinaai/jina-embeddings-v2-base-en_results'

In [168]:
generate_embeddings([words], tokenizer, model)

ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [107]:
df_vector_storage_time = pd.concat([df_vector_storage_time,df_vector_storage_time_helper], ignore_index=True)


C:\Users\hongk\AppData\Local\Temp\ipykernel_12668\1611950516.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_vector_storage_time = pd.concat([df_vector_storage_time,df_vector_storage_time_helper], ignore_index=True)


In [215]:
query_word = 'Cardiac Catheterization'
query_vector = generate_embeddings([query_word], tokenizer, model)
k = 5  # number of nearest neighbors to retrieve
D, I = index.search(query_vector, k)
# # Print results
print("Query word:", query_word)
print("Distances:", D)
print("Indices:", I)
print("Nearest neighbors:")
for idx in I[0]:
    print(df_list_of_documents['words'].iloc[idx])

AttributeError: 'numpy.int64' object has no attribute 'search'

In [136]:
for words in df_testing['words'].tolist()[0:5]:
    query = generate_embeddings([words], tokenizer, model)
    k = 3
    D, I = index.search(query, k)
    print(f"Top 5 most similar words to '{words}':")
    for i, (distance, index) in enumerate(zip(D[0], I[0])):
        print(f"{i+1}: {df_list_of_documents['words'][index]} (distance: {distance:.2f})")

Top 5 most similar words to 'Cardiac Catheterization':
1: Rehabilitation Program (distance: 564.20)
2: Cardiac Rehabilitation (distance: 564.00)
3: Temporal Arteritis Treatment (distance: 547.75)


AttributeError: 'numpy.int64' object has no attribute 'search'

In [106]:
df

,model_name,total_time
0,jinaai/jina-embeddings-v2-base-en,5.201945


In [79]:
query_word = 'Cardiac Catheterization'
query_vector = generate_embeddings([query_word], tokenizer, model)
k = 5  # number of nearest neighbors to retrieve
D, I = index.search(query_vector, k)
# # Print results
print("Query word:", query_word)
print("Distances:", D)
print("Indices:", I)
print("Nearest neighbors:")
for idx in I[0]:
    print(df_list_of_documents['words'].iloc[idx])

Query word: Cardiac Catheterization
Distances: [[582.13074 576.70593 559.2464  549.9591  539.26886]]
Indices: [[461 393 278 179  78]]
Nearest neighbors:
Mole Removal
Breast Reconstruction
Discharge Summary Fee
Osteoarthritis Treatment
Sports Medicine Consultation


In [60]:
import numpy as np
import pandas as pd
import faiss
from transformers import AutoModel, AutoTokenizer
import torch

# Step 1: Load the Jina embeddings model
model_name = 'jinaai/jina-embeddings-v2-base-en'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Function to generate embeddings
def generate_embeddings(texts, tokenizer, model):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).cpu().numpy()

# Step 2: Read words from the CSV file
csv_file = r"C:\Users\hongk\Desktop\embedding_test\embedding_trainign.xlsx" # update with your CSV file path
df_list_of_documents = pd.read_excel(csv_file)
words = df_list_of_documents['words'].tolist()

# # Step 3: Generate vectors for the words
vectors = generate_embeddings(words, tokenizer, model)

# # Step 4: Create the FAISS index
dimension = vectors.shape[1]  # dimension of vectors (determined by the model)
index = faiss.IndexFlatIP(dimension)  

# # Step 5: Add vectors to the index
index.add(vectors)

# Step 6: Perform a search
# For example, let's search for the nearest neighbors of the word 'example'
query_word = 'washing clothes'
query_vector = generate_embeddings([query_word], tokenizer, model)
k = 5  # number of nearest neighbors to retrieve
D, I = index.search(query_vector, k)

# # Print results
print("Query word:", query_word)
print("Distances:", D)
print("Indices:", I)
print("Nearest neighbors:")
for idx in I[0]:
    print(df_list_of_documents['words'].iloc[idx])

Some weights of BertModel were not initialized from the model checkpoint at jinaai/jina-embeddings-v2-base-en and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.1.output.dense.weight', 'encoder.layer.10.intermediate.dense.bias', 'encoder.layer.10.intermediate.dense.weight', 'encoder.layer.10.output.LayerNorm.bias', 'encoder.layer.10.output.LayerNorm.weight', 'encoder.layer.10.output.dense.bias', 'encoder.layer.10.output.dense.weight', 'encoder.layer.11.intermediate.dense.bias', 'encoder.layer.11.intermedi

Query word: washing clothes
Distances: [[473.67096 458.19995 450.40442 448.18982 422.47223]]
Indices: [[ 52  71  63  80 224]]
Nearest neighbors:
 Nursing Services
 Specialist Services
 Dietary Services
 Outpatient Services
 Ear Cleaning


In [61]:
dimension

768

In [7]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim

model = SentenceTransformer(
    "jinaai/jina-embeddings-v2-base-en", # switch to en/zh for English or Chinese
    trust_remote_code=True
)

# control your input sequence length up to 8192
model.max_seq_length = 1024

embeddings = model.encode([
    'How is the weather today?',
    'What is the current weather like today?'
])
print(cos_sim(embeddings[0], embeddings[1]))


c:\Users\hongk\Desktop\embedding_test\.conda\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
2024-06-13 10:39:54 | INFO | sentence_transformers.SentenceTransformer | Use pytorch device_name: cpu
2024-06-13 10:39:54 | INFO | sentence_transformers.SentenceTransformer | Load pretrained SentenceTransformer: jinaai/jina-embeddings-v2-base-en
c:\Users\hongk\Desktop\embedding_test\.conda\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Batches: 100%|██████████| 1/1 [00:00<00:00,  6.99it/s]

tensor([[0.9341]])


In [14]:
# read excel sheet 
list_of_documents = pd.read_excel(r"C:\Users\hongk\Desktop\embedding_test\embedding_trainign.xlsx")


In [15]:
db = FAI

,Room Charge
0,Private Room
1,Semi-private Room
2,General Ward
3,Intensive Care Unit (ICU)
4,High Dependency Unit (HDU)
...,...
288,Copy of Medical Records
289,Insurance Pre-Authorization
290,Billing Inquiry
291,Patient Education Materials


In [3]:
import torch
from transformers import AutoTokenizer, AutoModel

# Load tokenizer and model
model_name = "jinaai/jina-embeddings-v2-base-en"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Function to generate embeddings
def get_embeddings(text):
    # Tokenize input text
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    
    # Generate embeddings
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Extract the last hidden state
    embeddings = outputs.last_hidden_state
    return embeddings


Some weights of BertModel were not initialized from the model checkpoint at jinaai/jina-embeddings-v2-base-en and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.1.output.dense.weight', 'encoder.layer.10.intermediate.dense.bias', 'encoder.layer.10.intermediate.dense.weight', 'encoder.layer.10.output.LayerNorm.bias', 'encoder.layer.10.output.LayerNorm.weight', 'encoder.layer.10.output.dense.bias', 'encoder.layer.10.output.dense.weight', 'encoder.layer.11.intermediate.dense.bias', 'encoder.layer.11.intermedi

tensor([[[-0.1899,  1.1929, -0.4021,  ...,  0.6330, -0.5198, -0.8372],
         [-0.2069,  1.3158, -0.3773,  ...,  0.6481, -0.3240, -0.9079],
         [-0.1385,  1.3521, -0.2757,  ...,  0.7079, -0.5639, -0.9159],
         ...,
         [-0.2973,  1.0994, -0.3653,  ...,  0.6461, -0.1806, -0.7897],
         [-0.1881,  1.0875, -0.2531,  ...,  0.6337, -0.1190, -0.9248],
         [-0.1326,  1.0194, -0.1792,  ...,  0.5127, -0.3349, -0.9767]]])


7

In [8]:
! pip install fairseq

  Using cached fairseq-0.12.2.tar.gz (9.6 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached Cython-3.0.10-cp311-cp311-win_amd64.whl.metadata (3.2 kB)
  Using cached hydra_core-1.0.7-py3-none-any.whl.metadata (3.7 kB)
  Using cached omegaconf-2.0.6-py3-none-any.whl.metadata (3.0 kB)
  Using cached sacrebleu-2.4.2-py3-none-any.whl.metadata (58 kB)
  Using cached torch-2.3.1-cp311-cp311-win_amd64.whl.metadata (26 kB)
  Using cached torchaudio-2.3.1-cp311-cp311-win_amd64.whl.metadata (6.4 kB)
  Using cached antlr4_python3_runtime-4.8-py3-none-any.whl
  Using cached portalocker-2.8.2-py3-none-

DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [ ]:
!conda 

In [ ]:
!